**You must complete the cells with ### Your code goes here ###**


## **Transfer Learning**
Transfer learning in Convolutional Neural Networks (CNNs) is a technique where a model developed for a specific task is reused as the starting point for a model on a second, related task. This approach leverages the knowledge gained from the original task to improve the performance and training efficiency of the new task. Here’s how it works and why it’s useful:


**How Transfer Learning Works**

**Pre-trained Model:**

A CNN is first trained on a large dataset for a task such as image classification. Popular pre-trained models include VGG, ResNet, and Inception, which are trained on large datasets like ImageNet.

**Feature Extraction:**

The pre-trained model has already learned useful features from the initial dataset. For transfer learning, we use the weights and architecture of this pre-trained model and repurpose it for a new, related task.

**Fine-tuning:**

The final layers of the pre-trained model are either replaced or appended with new layers that are specific to the new task. Only these new layers, or sometimes the entire network, are retrained on the new dataset.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPool2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import time, datetime

**We load the cifar 10 dataset then separate the first 5 classes of data as the first training set.**
**Images with labels less than 5 are put in set 1, and the rest are put in set 2. Data is also normalized by dividing by 255.**

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 50000


X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0) #Dividing by 255 is for normalization
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


**Convert the lists to NumPy arrays and reshape them to the correct input format. Labels are one-hot encoded for classification.**

In [3]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

Y1_train = to_categorical(np.asarray(Y1_train), 5)
Y1_test = to_categorical(np.asarray(Y1_test), 5)

Y2_train = to_categorical(np.asarray(Y2_train), 10)
Y2_test = to_categorical(np.asarray(Y2_test), 10)
print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)

(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 5) (5000, 5)


**Split the training data into training and validation sets using an 80-20 split.**

In [4]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

**Build and compile Keras CNN model for classification.**

Your layers should be arranged like this:

Layer 1. Convolutional layer with 32 4x4 kernels, expecting input of shape (32, 32, 3) and using relu as activation function.

Layer 2. 2DMaxPooling layer  which pool_size is 2x2.

Layer 3. Convolutional layer with 32 4x4 kernels, expecting input of shape (32, 32, 3) and using relu as activation function.

Layer 4. 2DMaxPooling layer  which pool_size is 2x2.

Layer 5. Flatten layer


Layer 7. Fully connected layer with 256 neurons, using relu as activation function.

Layer 8. Fully connected layer with 5 neurons - acting as the final output classes.

In [5]:
### Your code goes here ###
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=(32, 32, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 29, 29, 32)          │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 32)          │          16,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         205,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 224,325 (876.27 KB)

 Trainable params: 224,325 (876.27 KB)

 Non-trainable params: 0 (0.00 B)

**Now it's time to train the network.**

In [6]:
start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
         epochs=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
200/200 - 24s - 120ms/step - accuracy: 0.5415 - loss: 1.1234 - val_accuracy: 0.6258 - val_loss: 0.9582
Epoch 2/10
200/200 - 40s - 199ms/step - accuracy: 0.6377 - loss: 0.9066 - val_accuracy: 0.6510 - val_loss: 0.8742
Epoch 3/10
200/200 - 40s - 201ms/step - accuracy: 0.6722 - loss: 0.8276 - val_accuracy: 0.6900 - val_loss: 0.8035
Epoch 4/10
200/200 - 43s - 213ms/step - accuracy: 0.7016 - loss: 0.7632 - val_accuracy: 0.6898 - val_loss: 0.7906
Epoch 5/10
200/200 - 40s - 202ms/step - accuracy: 0.7279 - loss: 0.7043 - val_accuracy: 0.7242 - val_loss: 0.7234
Epoch 6/10
200/200 - 20s - 100ms/step - accuracy: 0.7484 - loss: 0.6544 - val_accuracy: 0.7110 - val_loss: 0.7551
Epoch 7/10
200/200 - 22s - 110ms/step - accuracy: 0.7714 - loss: 0.6002 - val_accuracy: 0.7336 - val_loss: 0.7076
Epoch 8/10
200/200 - 41s - 203ms/step - accuracy: 0.7862 - loss: 0.5627 - val_accuracy: 0.7474 - val_loss: 0.6641
Epoch 9/10
200/200 - 21s - 104ms/step - accuracy: 0.8077 - loss: 0.5107 - val_accuracy: 

**The convolutional neural network for training a model for classes 5..9 by using the information gained by previous model is as follows:**


**Build and compile Follwed Keras CNN model for classification.**

Your layers should be arranged like this:

Layer 1-5: First 5 layers of the previous model (basically the convoluted layers) are used as it is in this model. Further they are made non-trainable to preserve the information gained by them while training of previous model. (So that means sense to freeze them.)

Layer 7: Fully connected layer with 128 neurons, using relu as activation function.

Layer 8. Fully connected layer with 10 neurons - acting as the final output classes.

In [7]:
### Your code goes here ###
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

for layer in model.layers[:5]:
    layer.trainable = False

new_model = Sequential()

for layer in model.layers[:5]:
    new_model.add(layer)

new_model.add(Dense(128, activation='relu'))

new_model.add(Dense(10, activation='softmax'))

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

new_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 29, 29, 32)          │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 32)          │          16,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 121,802 (475.79 KB)

 Trainable params: 103,818 (405.54 KB)

 Non-trainable params: 17,984 (70.25 KB)

**Now it's time to train the 2nd Part**

In [9]:
start = datetime.datetime.now()
hist2 = new_model.fit(x2_train, y2_train, epochs=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
200/200 - 7s - 34ms/step - accuracy: 0.7045 - loss: 0.8125 - val_accuracy: 0.7662 - val_loss: 0.6432
Epoch 2/10
200/200 - 7s - 36ms/step - accuracy: 0.7773 - loss: 0.6096 - val_accuracy: 0.7894 - val_loss: 0.5922
Epoch 3/10
200/200 - 6s - 28ms/step - accuracy: 0.7962 - loss: 0.5595 - val_accuracy: 0.7864 - val_loss: 0.5799
Epoch 4/10
200/200 - 7s - 36ms/step - accuracy: 0.8120 - loss: 0.5180 - val_accuracy: 0.8120 - val_loss: 0.5346
Epoch 5/10
200/200 - 10s - 50ms/step - accuracy: 0.8213 - loss: 0.4897 - val_accuracy: 0.8068 - val_loss: 0.5334
Epoch 6/10
200/200 - 9s - 44ms/step - accuracy: 0.8310 - loss: 0.4662 - val_accuracy: 0.8154 - val_loss: 0.5225
Epoch 7/10
200/200 - 10s - 51ms/step - accuracy: 0.8413 - loss: 0.4413 - val_accuracy: 0.8122 - val_loss: 0.5147
Epoch 8/10
200/200 - 13s - 65ms/step - accuracy: 0.8467 - loss: 0.4221 - val_accuracy: 0.8108 - val_loss: 0.5265
Epoch 9/10
200/200 - 9s - 46ms/step - accuracy: 0.8556 - loss: 0.3987 - val_accuracy: 0.8232 - val_lo